In [1]:
# !pip install git+git://github.com/jsignell/dask-geopandas.git

  Cloning git://github.com/jsignell/dask-geopandas.git to /tmp/pip-req-build-wmkzdt98
  Created wheel for dask-geopandas: filename=dask_geopandas-0+untagged.7.gcb73333-py3-none-any.whl size=13073 sha256=771843a4443b04c9cb4ebf5467f7d81c367acbcc1a432b4d69d2341add53952f
  Stored in directory: /tmp/pip-ephem-wheel-cache-qdqpagb6/wheels/ff/55/cb/2b4f9123cf14297d989da457790c5d863cda30fa41ab22d763
Successfully built dask-geopandas


In [2]:
from dask.distributed import Client, LocalCluster
import dask_geopandas as dg
import geopandas

In [3]:
client = Client(LocalCluster(n_workers=14))
client

Client Scheduler: tcp://127.0.0.1:45911 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 14 Cores: 28 Memory: 84.28 GB


In [4]:
df = geopandas.read_parquet("../../urbangrammar_samba/spatial_signatures/buildings/blg_0.pq")

In [5]:
ddf = dg.from_geopandas(df, npartitions=14)

In [6]:
def get_bbox(series):
    return series.apply(lambda g: g.minimum_rotated_rectangle)

In [7]:
bbox = ddf.geometry.map_partitions(get_bbox, meta=geopandas.GeoSeries())

In [8]:
ddf['area'] = ddf.geometry.area
ddf['length'] = ddf.geometry.length

In [9]:
ddf['eri'] = (ddf['area'] / bbox.area).pow(1./2) * (bbox.length / ddf['length'])

In [10]:
%%time
ddf.compute()

CPU times: user 1.54 s, sys: 189 ms, total: 1.73 s
Wall time: 6.62 s


,uID,geometry,area,length,eri
2326419,2326419,"POLYGON ((359111.400 434416.180, 359123.470 43...",131.44635,45.959465,0.999993
2326423,2326423,"POLYGON ((359130.950 434416.050, 359122.160 43...",64.71930,32.338801,1.000022
2326424,2326424,"POLYGON ((359130.610 434419.410, 359145.280 43...",158.77450,51.086096,1.000021
2326425,2326425,"POLYGON ((359160.490 434474.880, 359159.490 43...",189.31275,55.043887,0.999984
2326426,2326426,"POLYGON ((359156.180 434436.290, 359167.620 43...",163.25300,51.298783,0.999985
...,...,...,...,...,...
2573206,2573206,"POLYGON ((335422.740 427198.940, 335425.690 42...",532.25255,154.044586,0.599067
2573207,2573207,"POLYGON ((335421.370 427071.200, 335439.210 42...",314.15700,71.202935,1.000001
2573208,2573208,"POLYGON ((335432.380 427227.080, 335438.650 42...",192.07815,56.055383,0.999980
2573209,2573209,"POLYGON ((335464.150 427074.040, 335467.790 42...",402.79990,104.394290,0.997457


In [26]:
client.restart()

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


Client Scheduler: tcp://127.0.0.1:45425 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 7 Cores: 28 Memory: 84.28 GB


In [8]:
ddf = dg.read_parquet("../../urbangrammar_samba/spatial_signatures/buildings/blg_*.pq")